Naive RAG + ChatGPT gpt-4o-mini ( Dit prototype is in Pair programming gedaan door Pieter van der Werff & Jan Glazenborg)

In [1]:
# ! pip install openai
# ! pip install python-dotenv
# ! pip install pymupdf4llm
# ! pip install matplotlib
# ! pip install chromadb
# ! pip install llama-index
# ! pip install llama-index-vector-stores-chroma

from openai import OpenAI
import os 
from dotenv import load_dotenv
import pymupdf4llm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore


In [2]:

# Lezen document
# documents = pymupdf4llm.to_markdown("Leidraad kwaliteit AI in de zorg NL.pdf")
md_text = open("leidraad.txt")

# Chunking regels
chunk_size = 1000  # Size of each chunk
overlap = 200      # Number of overlapping characters

# API key en OpenAI laden
load_dotenv()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

Laden document

In [3]:
# --- Step 1: Load Document ---
# Read the plain text file
with open("leidraad.txt", "r", encoding="ISO-8859-1") as file:
    document = file.read()  # Read content into a string

ChromaDB aanmaken voor creeren vector database

In [4]:
# --- Step 2: Store Text in ChromaDB ---
# Initialize ChromaDB
chroma_client = chromadb.Client()
collection = chroma_client.create_collection("leidraad_ai_in_zorg")

Split in chunks

In [5]:

chunks = [
    document[i:i+chunk_size] 
    for i in range(0, len(document) - chunk_size + 1, chunk_size - overlap)
]




In [6]:

# Add chunks to ChromaDB
for idx, chunk in enumerate(chunks):
    collection.add(
        ids=[str(idx)],
        documents=[chunk]
    )

# Uncomment to print chunks for verification
# for idx, chunk in enumerate(chunks):
#     print(f"Chunk {idx}:\n{chunk}\n{'-'*80}")


User query

In [7]:
user_query = "Waar staat AIPA voor?"

Relevante chunks ophalen

In [8]:
# Retrieve relevant chunks using Chroma
results = collection.query(
    query_texts=[user_query],
    n_results=10
)

# Extract all top 10 relevant chunks
retrieved_texts = [doc[0] for doc in results['documents']]  # Assuming 'documents' is a list of lists

# Combine the texts into one string (optional)
top_10_retrieved_text = "\n".join(retrieved_texts)

# Print all top 10 chunks
print(top_10_retrieved_text)

n van risicos van het gebruik, inzetten en vergoeden van AIPAs. Ook bij een

verwachte lage impact van het AIPA op de patiënt, cliënt of burger blijft goed professioneel

handelen van belang, en blijven die delen van de leidraad die geen betrekking hebben op de

rechten en plichten van eindgebruikers als leidraad voor goed professioneel handelen van

toepassing. Dit is tenminste het geval als het AIPA geen deel uit gaat maken van een

hulpmiddel.

**Fasering**

De leidraad is ingedeeld in zes fasen:

   - Fase 1: Verzameling en beheer van de data

   - Fase 2: Ontwikkeling van het AIPA

   - Fase 3: Validatie van het AIPA

   - Fase 4: Ontwikkeling van de benodigde software

   - Fase 5: Effectbeoordeling van het AIPA in combinatie met de software

   - Fase 6: Implementatie en gebruik van het AIPA met software in de dagelijkse praktijk.

De daarin veronderstelde chronologie is niet bedoeld als dwingend, en past lang niet altijd op

de feitelijke of meest efficiënte volgorde van hand

Vraag antwoord op basis van top chunks

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": f"Je bent een assistent die vragen over de Leidraad AI in de zorg beantwoordt."},
        {"role": "user", "content": f"Baseer je antwoord op deze tekst: {top_10_retrieved_text}\n\n{user_query}"}
    ]
)

# --- Step 5: Output Result ---
print("Vraag:", user_query)
print("Antwoord:", response.choices[0].message.content)

Vraag: Wat betekent AIPA?
Antwoord: De afkorting AIPA staat niet specifiek gedefinieerd in de tekst die je hebt verstrekt. Maar in de context van de zorg en AI zou het kunnen staan voor "Artificial Intelligence Powered Application", een toepassing die door kunstmatige intelligentie wordt aangestuurd. Raadpleeg de originele leidraad voor een precieze definitie, omdat de betekenis van afkortingen kan variëren afhankelijk van de context.
